In [ ]:
%%time
from src.data.make_dataset import load_list, get_stock_data
from src.models.StockModel import StockModel
window_size = 20
start_date = '2017-01-01'
end_date = '2024-09-01'
feature_columns = ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD']
target = "Open"

# Load symbols
nasdaq_symbols = load_list("NASDAQ")
sp500_symbols = load_list("SP500")

# Test tickers, sp500 symbols not also in nasdaq
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:20]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC']
train_tickers = ['^IXIC'] + nasdaq_symbols
train_tickers = train_tickers[:26]

# Download data
combined_data = get_stock_data(train_tickers, "1d", start_date, end_date)
combined_data.info()
# Test data
test_data = get_stock_data(test_tickers, "1d", start_date, end_date)

layer_config = [(1, 128),(1,256),(2,128),(3,128),(3,256)]
for i in layer_config:
    print(f"XXXXXXXXXXXXXXXX Running {i[0]} {i[1]} layers XXXXXXXXXXXXXXXXXXXX")
    # Create and train model
    stock_model = StockModel(window_size=window_size, feature_columns=feature_columns, target_name=target, export=True)
    
    stock_model.train(combined_data, patience=10, epochs=150, graph=False, layers=i[0], units_per_layer=i[1])
    metrics_dict, mean_metrics = stock_model.evaluate_many(test_data, graph=False)
    print(metrics_dict)
    print(mean_metrics)

In [ ]:
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:5]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC', '^DJI']
# Test data
test_data = get_stock_data(["^GSPC", "^DJI"], "1d", start_date, end_date)

metrics_dict, mean_metrics = stock_model.evaluate_many(test_data, graph=True)
print(metrics_dict)
print(mean_metrics)
